# End-to-End Mass Regression for Boosted Top Quarks 


## Importing the Needed Libraries

In [1]:
import numpy as np
import os, glob
import time
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import *

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rcParams["figure.figsize"] = (5,5)
plt.switch_backend('agg')
%matplotlib inline

import random
import gc

##Defining the Resnet-15 Model:

In [2]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(ResBlock, self).__init__()
        self.downsample = out_channels//in_channels
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=self.downsample, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=self.downsample)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)

        if self.downsample > 1:
            residual = self.shortcut(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, in_channels, nblocks, fmaps):
        super(ResNet, self).__init__()
        self.fmaps = fmaps
        self.nblocks = nblocks

        self.conv0 = nn.Conv2d(in_channels, fmaps[0], kernel_size=7, stride=1, padding=1)
        self.layer1 = self.block_layers(self.nblocks, [fmaps[0],fmaps[0]])
        self.layer2 = self.block_layers(1, [fmaps[0],fmaps[1]])
        self.layer3 = self.block_layers(self.nblocks, [fmaps[1],fmaps[1]])

        # no FC
        self.fc = nn.Linear(self.fmaps[1]+2, 1)

        self.GlobalMaxPool2d = nn.AdaptiveMaxPool2d((1,1))
        
    def block_layers(self, nblocks, fmaps):
        layers = []
        for _ in range(nblocks):
            layers.append(ResBlock(fmaps[0], fmaps[1]))
        return nn.Sequential(*layers)

    def forward(self, X):

        x = self.conv0(X[0])
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.GlobalMaxPool2d(x)
        x = x.view(x.size()[0], self.fmaps[1])
        # concat with seed pos
        x = torch.cat([x, X[1], X[2]], 1)
        # FC
        x = self.fc(x)
        return x

##Defining the Loss Function

In [3]:
def mae_loss_wgtd(pred, true, is_cuda, wgt=1.):
    loss = wgt*(pred-true).abs()
    if is_cuda: 
        loss = loss.cuda()
    return loss.mean()

##Define the Dataloaders

In [4]:
def transform_y(y, m0_scale = 500):
    return y/m0_scale

def inv_transform(y, m0_scale = 500):
    return y*m0_scale

class ParquetDataset(Dataset):
    def __init__(self, filename, label):
        self.parquet = pq.ParquetFile(filename)
        self.cols = ["X_jet","genM","iphi","ieta"]
        self.label = label
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index,columns=self.cols).to_pandas()
        data_dict={}
        data_dict['X_jet'] = np.array([[[coord for coord in xk] for xk in xj] for xj in data["X_jet"].values[0]], ndmin=3,dtype=np.float32) 
        data_dict['X_jet'][0] = channel1_scale * data_dict['X_jet'][0] 
        data_dict['X_jet'][1] = channel2_scale * data_dict['X_jet'][1] 
        data_dict['X_jet'][2] = channel3_scale   * data_dict['X_jet'][2] 
        data_dict['genM'] = transform_y(np.float32(data['genM'].values))
        data_dict['iphi'] = np.float32(data['iphi'].values)/360.
        data_dict['ieta'] = np.float32(data['ieta'].values)/170.
        # Zero-Suppression
        data_dict['X_jet'][data_dict['X_jet'] < 1.e-3] = 0. 
        # High Value Suppression
        data_dict['X_jet'][0][data_dict['X_jet'][0] > 50] = 1. 
        data_dict['X_jet'][1][data_dict['X_jet'][1] > 5] = 1. 
        data_dict['X_jet'][2][data_dict['X_jet'][2] > 5] = 1. 
        return data_dict
        
    def __len__(self):
        return self.parquet.num_row_groups
       
def train_val_loader(datasets, batch_size, random_sampler=True):
    dset = ConcatDataset([ParquetDataset(dataset, datasets.index(dataset)) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    print(len(dset))
    train_cut = int(len(dset) * 0.9)
    val_cut = int(len(dset) * 0.05)
    if random_sampler: 
        train_sampler = SubsetRandomSampler(idxs[:train_cut])
        val_sampler = SubsetRandomSampler(idxs[train_cut:train_cut+val_cut])
        test_sampler = SubsetRandomSampler(idxs[train_cut+val_cut:])
    else: 
        train_sampler, val_sampler = None, None 
    train_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=4, sampler=train_sampler, pin_memory=True)
    val_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=4, sampler=val_sampler, pin_memory=True)
    test_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=4, sampler=test_sampler, pin_memory=True)
    return train_loader, val_loader,test_loader

##Defining Train and Evaluation Utilities

In [5]:
def logger(s, f, run_logger=True):
    print(s)
    if run_logger:
        f.write('%s\n' % str(s))
        
def load_model(model_name, resnet, optimizer, lr_scheduler):
    print(model_name)
    checkpoint = torch.load(os.path.join(params["save_path"], "MODELS") + "/%s/%s " % (params["expt_name"], model_name))
    resnet.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return resnet, optimizer, lr_scheduler

def do_eval(resnet, val_loader, epoch):
    mass_bins = np.arange(85, 500, 25)  # for histogram in eval()
    loss_ = 0.
    m_pred_, m_true_, mae_, mre_ = [], [], [], []
    now = time.time()
    ma_low = transform_y(85.0)  # convert from GeV to network units
    for i, data in enumerate(val_loader):
        X, m0, iphi, ieta = data['X_jet'].cuda(), data['genM'].cuda(), data['iphi'].cuda(), data['ieta'].cuda()
        X = X[m0[:, 0] > ma_low]
        iphi = iphi[m0[:, 0] > ma_low]
        ieta = ieta[m0[:, 0] > ma_low]
        m0 = m0[m0[:, 0] > ma_low]
        logits = resnet([X, iphi, ieta])
        loss_ += mae_loss_wgtd(logits, m0, is_cuda=params["is_cuda"]).item()

        # Undo preprocessing on mass
        logits, m0 = inv_transform(logits), inv_transform(m0)
        mae = (logits - m0).abs()
        mre = ((logits - m0).abs() / m0)
        # Store batch metrics:

        m_pred_.append(logits.tolist())
        m_true_.append(m0.tolist())
        mae_.append(mae.tolist())
        mre_.append(mre.tolist())

        gc.collect()

    now = time.time() - now
    m_true_ = np.concatenate(m_true_)
    m_pred_ = np.concatenate(m_pred_)
    mae_ = np.concatenate(mae_)
    mre_ = np.concatenate(mre_)

    logger('%d: Val m_pred: %s... ' % (epoch, str(np.squeeze(m_pred_[:5]))), f, params["run_logger"])
    logger('%d: Val m_true: %s... ' % (epoch, str(np.squeeze(m_true_[:5]))), f, params["run_logger"])
    logger('%d: Val time:%.2fs in %d steps for N=%d ' % (epoch, now, len(val_loader), len(m_true_)), f,
           params["run_logger"])
    logger('%d: Val loss:%f, mae:%f, mre:%f ' % (epoch, loss_ / len(val_loader), np.mean(mae_), np.mean(mre_)), f,
           params["run_logger"])

    score_str = 'epoch%d_mae%.4f ' % (epoch, np.mean(mae_))

    # Check 1D m_pred
    hst = np.histogram(np.squeeze(m_pred_), bins=mass_bins)[0]
    logger('%d: Val m_pred, [85,500,25] MeV: %s ' % (epoch, str(np.uint(hst))), f, params["run_logger"])
    mlow = hst[0]
    mrms = np.std(hst)
    logger('%d: Val m_pred, [85,500,25] MeV: low:%d, rms: %f ' % (epoch, mlow, mrms), f, params["run_logger"])
    plt.hist(m_true_, range=(80, 510), bins=20, histtype='step', label=r'$\mathrm{m_{true}}$', linestyle='--',
             color='grey', alpha=0.6)
    plt.hist(m_pred_, range=(80, 510), bins=20, histtype='step', label=r'$\mathrm{m_{pred}}$', linestyle='--',
             color='C0', alpha=0.6)
    plt.xlim(80, 510)
    plt.xlabel(r'$\mathrm{m}$', size=16)
    plt.legend(loc='upper right')
    plt.savefig(
        os.path.join(params["save_path"], "PLOTS") + '/%s/test_mpred_%s.png' % (params["expt_name"], score_str),
        bbox_inches='tight')
    plt.close()
    return mre_, m_pred_, m_true_, np.mean(mae_)


def train(resnet, optimizer, lr_scheduler, epochs, train_loader, val_loader):
    if params["load_epoch"] != 0:
        model_name = 'Tops_ResNet_blocks_3_model_epoch_%d ' % (params["load_epoch"])
        resnet, optimizer, lr_scheduler = load_model(model_name, resnet, optimizer, lr_scheduler)
    print_step = 100
    resnet.train()
    for e in range(epochs):
        resnet.train()
        global f
        f = open(os.path.join(params["save_path"], "LOGS") + '/%s.log ' % (params["expt_name"]), 'a')
        epoch = e + 1 + params["load_epoch"]
        n_trained = 0
        loss_ = 0.
        logger('>> Epoch %d <<<<<<<<' % (epoch), f)

        # Run training
        resnet.train()
        now = time.time()
        for i, data in enumerate(train_loader):
            X, m0, iphi, ieta = data['X_jet'].cuda(), data['genM'].cuda(), data['iphi'].cuda(), data['ieta'].cuda()
            optimizer.zero_grad()
            logits = resnet([X, iphi, ieta])
            loss = mae_loss_wgtd(logits, m0, is_cuda=params["is_cuda"])
            loss.backward()
            loss_ += loss.item()
            optimizer.step()

            n_trained += 1

            if i % print_step == 0:
                logits, m0 = inv_transform(logits), inv_transform(m0)
                mae = (logits - m0).abs().mean()
                mre = ((logits - m0).abs() / m0).mean()
                logger('%d: (%d/%d) m_pred: %s' % (epoch, i, len(train_loader), str(np.squeeze(logits.tolist()[:5]))),
                       f, params["run_logger"])
                logger('%d: (%d/%d) m_true: %s' % (epoch, i, len(train_loader), str(np.squeeze(m0.tolist()[:5]))), f,
                       params["run_logger"])
                logger('%d: (%d/%d) Train loss: %f, mae: %f,mre: %f' % (
                    epoch, i, len(train_loader), loss.item(), mae.item(), mre.item()), f, params["run_logger"])
            gc.collect()

        now = time.time() - now
        logits, m0 = inv_transform(logits), inv_transform(m0)
        mae = (logits - m0).abs().mean()
        mre = ((logits - m0).abs() / m0).mean()
        logger('%d: Train time: %.2fs in %d steps for N:%d' % (epoch, now, len(train_loader), n_trained), f,
               params["run_logger"])
        logger('%d: Average Train loss: %f, Final mae: %f,Final mre: %f' % (
            epoch, loss_ / len(train_loader), mae.item(), mre.item()), f, params["run_logger"])

        gc.collect()

        # Run Validation
        resnet.eval()
        _, _, _, val_loss = do_eval(resnet, val_loader, epoch)
        lr_scheduler.step(val_loss)
        gc.collect()
        torch.save({
            'epoch': e,
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': lr_scheduler.state_dict()
        }, os.path.join(params["save_path"], "MODELS") + '/%s/Tops_ResNet_blocks_3_model_epoch_%d' % (
            params["expt_name"], epoch))

        if params["run_logger"]:
            f.close()

## Starting the Training Process
Let's start by defining the hyperparameters:

In [6]:
params={
  "batch_size": 1024,
  "epochs": 50,
  "load_epoch": 0,
  "lr": 1e-3,
  "resblocks": 3,
  "input_channels": 3,
  "fmaps": [
    16,
    32
  ],
  "is_cuda": 1,
  "run_logger": 1,
  "expt_name": "TopGun_scaled-target&input-500-0.02-0.2-1_lr_scheduled-1e-3",
  "save_path": ".",
  "data_path": ".",
  "channel1_scale": 0.02,
  "channel2_scale": 0.2,
  "channel3_scale": 1.0,
  "seed": 0
}

In [ ]:
np.random.seed(params["seed"])
torch.manual_seed(params["seed"])
random.seed(params["seed"])

LOG_PATH = os.path.join(params["save_path"], "LOGS")
MODEL_PATH = os.path.join(params["save_path"], "MODELS")
PLOT_PATH = os.path.join(params["save_path"], "PLOTS")

if not os.path.isdir(LOG_PATH):
    os.makedirs(LOG_PATH)
for d in [MODEL_PATH, PLOT_PATH]:
    if not os.path.isdir('%s/%s' % (d, params["expt_name"])):
        os.makedirs('%s/%s' % (d, params["expt_name"]))

train_loader, val_loader, test_loader = train_val_loader(
    [os.path.join(params["data_path"], f) for f in
     os.listdir(params["data_path"])], params["batch_size"])

resnet = ResNet(params["input_channels"], params["resblocks"], params["fmaps"])

if params["is_cuda"]:
    resnet.cuda()

optimizer = optim.Adam(resnet.parameters(), lr=params["lr"])
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
train(resnet, optimizer, lr_scheduler, params["epochs"], train_loader, val_loader)

## Evaluating on Validation Data

In [ ]:
np.random.seed(params["seed"])
torch.manual_seed(params["seed"])
random.seed(params["seed"])

LOG_PATH = os.path.join(params["save_path"], "LOGS")
MODEL_PATH = os.path.join(params["save_path"], "MODELS")
PLOT_PATH = os.path.join(params["save_path"], "PLOTS")

if not os.path.isdir(LOG_PATH):
    os.makedirs(LOG_PATH)
for d in [MODEL_PATH, PLOT_PATH]:
    if not os.path.isdir('%s/%s' % (d, params["expt_name"])):
        os.makedirs('%s/%s' % (d, params["expt_name"]))

train_loader, val_loader, test_loader = train_val_loader(
    [os.path.join(params["data_path"], f) for f in
     os.listdir(params["data_path"])], params["batch_size"])

resnet = ResNet(params["input_channels"], params["resblocks"], params["fmaps"])

if params["is_cuda"]:
    resnet.cuda()

optimizer = optim.Adam(resnet.parameters(), lr=params["lr"])

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

if load_epoch != 0:
        model_name = 'Tops_ResNet_blocks_3_model_epoch_%d'%(params["load_epoch"])
        resnet, optimizer,lr_scheduler = load_model(model_name, resnet, optimizer,lr_scheduler)
resnet.eval()
mre,m_pred,m_true,mean_mae=do_eval(resnet, val_loader, params["load_epoch"])

In [ ]:
df=pd.DataFrame()
df["m_pred"]=m_pred[:,0]
df["mre"]=mre[:,0]
df["m_true"]=m_true[:,0]
df_select=df[df["mre"]>0.5]
plt.title("Percentage of samples with mre bigger than 50%%: %.4f%%"%(df_select.shape[0]*100/df.shape[0]))
plt.scatter(df_select["m_true"],df_select["mre"])
plt.xlabel("Target Mass")
plt.ylabel("MRE")
plt.savefig(os.path.join(params["save_path"], "PLOTS") +'%s/mpred_massvsmre_val.png'%(params['expt_name']), bbox_inches='tight')

## Evaluating on Test Data

In [ ]:
np.random.seed(params["seed"])
torch.manual_seed(params["seed"])
random.seed(params["seed"])

LOG_PATH = os.path.join(params["save_path"], "LOGS")
MODEL_PATH = os.path.join(params["save_path"], "MODELS")
PLOT_PATH = os.path.join(params["save_path"], "PLOTS")

if not os.path.isdir(LOG_PATH):
    os.makedirs(LOG_PATH)
for d in [MODEL_PATH, PLOT_PATH]:
    if not os.path.isdir('%s/%s' % (d, params["expt_name"])):
        os.makedirs('%s/%s' % (d, params["expt_name"]))

train_loader, val_loader, test_loader = train_val_loader(
    [os.path.join(params["data_path"], f) for f in
     os.listdir(params["data_path"])], params["batch_size"])

resnet = ResNet(params["input_channels"], params["resblocks"], params["fmaps"])

if params["is_cuda"]:
    resnet.cuda()

optimizer = optim.Adam(resnet.parameters(), lr=params["lr"])

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

if load_epoch != 0:
        model_name = 'Tops_ResNet_blocks_3_model_epoch_%d'%(params["load_epoch"])
        resnet, optimizer = load_model(model_name, resnet, optimizer,lr_scheduler)
resnet.eval()
mre,m_pred,m_true,mean_mae=do_eval(resnet, val_loader, params["load_epoch"])

In [ ]:
df=pd.DataFrame()
df["m_pred"]=m_pred[:,0]
df["mre"]=mre[:,0]
df["m_true"]=m_true[:,0]
df_select=df[df["mre"]>0.5]
plt.title("Percentage of samples with mre bigger than 50%%: %.4f%%"%(df_select.shape[0]*100/df.shape[0]))
plt.scatter(df_select["m_true"],df_select["mre"])
plt.xlabel("Target Mass")
plt.ylabel("MRE")
plt.savefig(os.path.join(params["save_path"], "PLOTS") +'%s/mpred_massvsmre_test.png'%(params['expt_name']), bbox_inches='tight')